In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Conv2D, MaxPooling2D, Flatten, MaxPooling1D, Conv1D
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf

import tensorflow_addons as tfa
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df=pd.read_csv('input10.csv')

In [3]:
df.head()

,comment,class
0,FIXME formatters are not thread safe,DEFECT
1,"XXX Jon Skeet The comment ""if it hasn't bee...",DEFECT
2,"I hate to admit it, but we don't know what h...",DEFECT
3,Just a note StarTeam has a status for NEW wh...,DEFECT
4,the generated classes must not be added in t...,DEFECT


In [4]:
df['class'].value_counts()

DESIGN            2703
IMPLEMENTATION     757
DEFECT             472
TEST                85
DOCUMENTATION       54
Name: class, dtype: int64

In [5]:
X = df['comment']
Y = df['class']
le = LabelEncoder()
Y = le.fit_transform(Y)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(Y))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)
print("\n Class weights:")
#print(np.unique(Y, return_counts = True))
weight_dict = {}
counts = np.unique(Y, return_counts = True)[1]
total = np.sum(counts)
for i in range (0, 5):
    weight_dict[i] = (total - counts[i]) / counts[i]

print(weight_dict)

Y = Y.reshape(-1,1)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DEFECT', 'DESIGN', 'DOCUMENTATION', 'IMPLEMENTATION', 'TEST']

 Class weights:
{0: 7.625, 1: 0.5061043285238623, 2: 74.38888888888889, 3: 4.3778071334214, 4: 46.89411764705882}


In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20, stratify=Y, random_state=42)

max_words = 10000
max_len = 250
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
train_sequences = tok.texts_to_sequences(X_train)
train_sequences_matrix = pad_sequences(train_sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [7]:
print(len(train_sequences_matrix[0]))

250


In [11]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(128)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(5,name='out_layer')(layer)
    layer = Activation('softmax')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [12]:
model = RNN()
model.summary()
#opt = tf.keras.optimizers.Adadelta(learning_rate=0.1)
#opt = tf.keras.optimizers.Adam(learning_rate=0.001)
opt = tfa.optimizers.AdaBelief(learning_rate=5e-3)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['sparse_categorical_accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 250)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 250, 50)           500000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               91648     
                                                                 
 FC1 (Dense)                 (None, 256)               33024     
                                                                 
 activation_2 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 5)                 1285

In [13]:
#model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_filepath,
                save_weights_only=True,
                monitor='val_loss',
                mode='min',
                save_best_only=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=0.000001)

model.fit(train_sequences_matrix,Y_train,batch_size=128,epochs=50,validation_split=0.2, callbacks=[model_checkpoint_callback,reduce_lr], class_weight=weight_dict)

model.load_weights(checkpoint_filepath)

Epoch 1/50
21/21 [==============================] - 3s 65ms/step - loss: 6.1152 - sparse_categorical_accuracy: 0.1425 - val_loss: 1.6321 - val_sparse_categorical_accuracy: 0.1718 - lr: 0.0050
Epoch 2/50
21/21 [==============================] - 1s 39ms/step - loss: 6.0335 - sparse_categorical_accuracy: 0.1563 - val_loss: 2.2445 - val_sparse_categorical_accuracy: 0.1948 - lr: 0.0050
Epoch 3/50
21/21 [==============================] - 1s 40ms/step - loss: 5.9271 - sparse_categorical_accuracy: 0.1390 - val_loss: 1.8500 - val_sparse_categorical_accuracy: 0.1380 - lr: 0.0050
Epoch 4/50
21/21 [==============================] - 1s 40ms/step - loss: 5.6340 - sparse_categorical_accuracy: 0.1217 - val_loss: 1.8844 - val_sparse_categorical_accuracy: 0.2055 - lr: 0.0050
Epoch 5/50
21/21 [==============================] - 1s 38ms/step - loss: 4.7107 - sparse_categorical_accuracy: 0.1628 - val_loss: 1.8218 - val_sparse_categorical_accuracy: 0.2500 - lr: 0.0050
Epoch 6/50
21/21 [======================

In [14]:
model.save("LSTM_imbalanced_multi.h5")

In [15]:
accr = model.evaluate(test_sequences_matrix,Y_test)

26/26 [==============================] - 0s 15ms/step - loss: 1.0722 - sparse_categorical_accuracy: 0.5816


In [16]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.072
  Accuracy: 0.582


In [17]:
y_pred = model.predict(test_sequences_matrix)

26/26 [==============================] - 1s 14ms/step


In [18]:
y_pred2 = [np.argmax(x) for x in y_pred]

In [19]:
print(classification_report(Y_test, y_pred2, digits=6, target_names=multiclass_labels))

                precision    recall  f1-score   support

        DEFECT   0.393939  0.553191  0.460177        94
        DESIGN   0.830688  0.580407  0.683351       541
 DOCUMENTATION   0.146341  0.545455  0.230769        11
IMPLEMENTATION   0.406780  0.631579  0.494845       152
          TEST   0.214286  0.352941  0.266667        17

      accuracy                       0.581595       815
     macro avg   0.398407  0.532715  0.427162       815
  weighted avg   0.679160  0.581595  0.607654       815

